<a href="https://colab.research.google.com/github/andry2327/Machine-Learning-and-Deep-Learning-LAB--Google-Colab---PoliTo/blob/main/L06/es1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet
---

In [73]:
import torch
import torchvision
from torchvision import transforms as T
import torch.nn.functional as F
import torch.nn as nn

from torchvision.models import resnet18

In [74]:
is_pretrained=True

In [75]:
ResNet = resnet18(weights='DEFAULT', ) # pretrained=is_pretrained is 
# Replace the final fully connected layer
ResNet.fc = nn.Linear(512, 10)

Data Loader

In [76]:
def get_data(batch_size, test_batch_size=256):
    # Prepare data transformations and then combine them sequentially
    transform = list()
    transform.append(T.Resize((227,227)))
    transform.append(T.ToTensor()) # Converts Numpy to Pytorch Tensor
    transform.append(T.Normalize(mean=[0.5], std=[0.5])) # Normalizes the Tensors between [-1, 1]
    transform = T.Compose(transform) # Composes the above transformations into one.
    # Load data
    full_training_data = torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True)
    test_data = torchvision.datasets.CIFAR10('./data', train=False, transform=transform, download=True)
    # Create train and validation splits
    num_samples = len(full_training_data)
    training_samples = int(num_samples*0.8+1)
    validation_samples = num_samples - training_samples
    training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples,
    validation_samples])
    # Initialize dataloaders
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

In [77]:
train_set, val_set, test_set = get_data(128)

Files already downloaded and verified
Files already downloaded and verified


In [78]:
# Set the model to evaluation mode
ResNet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [79]:
# Define the classes for CIFAR10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Testing: pre_trained=TRUE

In [80]:
if torch.cuda.is_available():
  ResNet.cuda()
  ResNet.to('cuda:0')

correct = 0
total = 0
idx = 0

for data in test_set:

  imgs, labels = data
  if torch.cuda.is_available():
    imgs = imgs.cuda()
    labels = labels.cuda()

  # predict ResNet output
  outputs = ResNet(imgs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()

  idx+=1
  print(f'DONE {idx}/{len(test_set)} --- {round(100*idx/len(test_set), 4)}%')


DONE 1/40 --- 2.5%
DONE 2/40 --- 5.0%
DONE 3/40 --- 7.5%
DONE 4/40 --- 10.0%
DONE 5/40 --- 12.5%
DONE 6/40 --- 15.0%
DONE 7/40 --- 17.5%
DONE 8/40 --- 20.0%
DONE 9/40 --- 22.5%
DONE 10/40 --- 25.0%
DONE 11/40 --- 27.5%
DONE 12/40 --- 30.0%
DONE 13/40 --- 32.5%
DONE 14/40 --- 35.0%
DONE 15/40 --- 37.5%
DONE 16/40 --- 40.0%
DONE 17/40 --- 42.5%
DONE 18/40 --- 45.0%
DONE 19/40 --- 47.5%
DONE 20/40 --- 50.0%
DONE 21/40 --- 52.5%
DONE 22/40 --- 55.0%
DONE 23/40 --- 57.5%
DONE 24/40 --- 60.0%
DONE 25/40 --- 62.5%
DONE 26/40 --- 65.0%
DONE 27/40 --- 67.5%
DONE 28/40 --- 70.0%
DONE 29/40 --- 72.5%
DONE 30/40 --- 75.0%
DONE 31/40 --- 77.5%
DONE 32/40 --- 80.0%
DONE 33/40 --- 82.5%
DONE 34/40 --- 85.0%
DONE 35/40 --- 87.5%
DONE 36/40 --- 90.0%
DONE 37/40 --- 92.5%
DONE 38/40 --- 95.0%
DONE 39/40 --- 97.5%
DONE 40/40 --- 100.0%


In [81]:
# Compute the accuracy
accuracy = 100 * correct / total

# Print the accuracy
print('Accuracy of the ResNet-18 model on the CIFAR10 test set (DEFAULT WEIGTHS): %.2f%%' % accuracy)


Accuracy of the ResNet-18 model on the CIFAR10 test set (DEFAULT WEIGTHS): 8.83%


### Training and Testing: pre_trained=False

In [82]:
def get_loss_function():
  loss_function = torch.nn.CrossEntropyLoss()
  return loss_function

In [83]:
def get_optimizer(net, lr, wd, momentum):
    optimizer = torch.optim.SGD(params=net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
    return optimizer

In [84]:
def train(net, data_loader, optimizer, cost_function, device='cuda:0'):
    samples = 0.
    cumulative_loss = 0.
    cumulative_accuracy = 0.
    net.train() # Strictly needed if network contains layers which has different behaviours between train and test
    for batch_idx, (inputs, targets) in enumerate(data_loader):
        # Load data into GPU
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs) # Forward pass
        loss = cost_function(outputs, targets) # Apply the loss
        loss.backward() # Backward pass
        optimizer.step() # Update parameters
        optimizer.zero_grad() # Reset the optimizer
        samples += inputs.shape[0]
        cumulative_loss += loss.item()
        _, predicted = outputs.max(1)
        cumulative_accuracy += predicted.eq(targets).sum().item()
    return cumulative_loss/samples, cumulative_accuracy/samples*100

In [85]:
def test(net, data_loader, cost_function, device='cuda:0'):
    samples = 0.
    cumulative_loss = 0.
    cumulative_accuracy = 0.
    net.eval() # Strictly needed if network contains layers which have different behaviours between train and test
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            # Load data into GPU
            samples += len(inputs)
            inputs, targets = inputs.to(device), targets.to(device)
            # Forward pass
            outputs = net(inputs)
            _, predicted = outputs.max(1)
            cumulative_accuracy += predicted.eq(targets).sum().item()
    return cumulative_loss/samples, cumulative_accuracy/samples*100

In [86]:
def main(batch_size=128, device='cuda:0', learning_rate=0.05, weight_decay=0.000001, momentum=0.9, epochs=20):
    train_loader, val_loader, test_loader = get_data(batch_size)
    # TODO for defining AlexNet
    net = ResNet.to(device)
    optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
    loss_function = get_loss_function()

    for e in range(epochs):

        train_loss, train_accuracy = train(net, train_loader, optimizer, loss_function)
        val_loss, val_accuracy = test(net, val_loader, loss_function)

        print('Epoch: {:d}'.format(e+1))
        print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
        print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
        print('-----------------------------------------------------')
        print('After training:')

        train_loss, train_accuracy = test(net, train_loader, loss_function)
        val_loss, val_accuracy = test(net, val_loader, loss_function)
        test_loss, test_accuracy = test(net, test_loader, loss_function)

        print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
        print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
        print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
        print('-----------------------------------------------------')

In [87]:
main()

Files already downloaded and verified
Files already downloaded and verified
Epoch: 1
	 Training loss 0.00517, Training accuracy 78.84
	 Validation loss 0.00000, Validation accuracy 81.17
-----------------------------------------------------
After training:
	 Training loss 0.00000, Training accuracy 83.26
	 Validation loss 0.00000, Validation accuracy 81.17
	 Test loss 0.00000, Test accuracy 80.58
-----------------------------------------------------
Epoch: 2
	 Training loss 0.00214, Training accuracy 90.93
	 Validation loss 0.00000, Validation accuracy 87.00
-----------------------------------------------------
After training:
	 Training loss 0.00000, Training accuracy 91.19
	 Validation loss 0.00000, Validation accuracy 87.00
	 Test loss 0.00000, Test accuracy 86.41
-----------------------------------------------------
Epoch: 3
	 Training loss 0.00131, Training accuracy 94.23
	 Validation loss 0.00000, Validation accuracy 90.08
-----------------------------------------------------
Aft